# 🎯 Wikipedia Link Prediction - Complete Pipeline

Production-ready pipeline for predicting Wikipedia internal links.

**Components:**
- Fast phrase extraction (n-grams matching anchor dictionary)
- V6 linkability-based scoring (F1 ≈ 0.52)
- Semantic disambiguation for ambiguous anchors
- Wikipedia-style HTML output for comparison

**Usage:**
1. Run all cells to initialize
2. Evaluate on test set
3. Test on specific articles
4. Generate side-by-side HTML comparisons

## 1. Setup & Configuration

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import pickle
import re
import html
import time
from pathlib import Path
from typing import List, Dict, Set, Tuple, Optional
from dataclasses import dataclass
from collections import defaultdict

import numpy as np
import polars as pl
import torch
import requests
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [ ]:
@dataclass
class Config:
    """Pipeline configuration - best parameters from experiments."""
    # Paths
    parquet_path: str = "articles_fr_merged.parquet"
    mapping_cache: str = "url_to_id_mapping.pkl"
    anchor_cache: str = "anchor_dictionary.pkl"
    
    # Qdrant
    qdrant_host: str = "localhost"
    qdrant_port: int = 6333
    collection_name: str = "wikipedia_fr"
    
    # Model
    embedding_model: str = "intfloat/multilingual-e5-large"
    
    # Scoring (V6 best params)
    score_threshold: float = 3.5
    max_links_per_article: int = 500
    
    # Disambiguation
    use_semantic_disambiguation: bool = True
    disambiguation_threshold: int = 5
    
    # Extraction
    max_ngram: int = 5
    
    # Device
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

CONFIG = Config()

print("="*70)
print("🎯 WIKIPEDIA LINK PREDICTION PIPELINE")
print("="*70)
print(f"   Device: {CONFIG.device}")
print(f"   Threshold: {CONFIG.score_threshold}")
print(f"   Disambiguation: {CONFIG.use_semantic_disambiguation}")

In [ ]:
# Stop words - function words that should never be linked
STOP_WORDS = frozenset({
    'le', 'la', 'les', 'un', 'une', 'des', 'du', 'de', 'au', 'aux',
    'ce', 'cette', 'ces', 'mon', 'ma', 'mes', 'ton', 'ta', 'tes',
    'son', 'sa', 'ses', 'notre', 'nos', 'votre', 'vos', 'leur', 'leurs',
    'je', 'tu', 'il', 'elle', 'on', 'nous', 'vous', 'ils', 'elles',
    'qui', 'que', 'quoi', 'dont', 'où', 'et', 'ou', 'mais', 'donc',
    'car', 'ni', 'si', 'dans', 'pour', 'par', 'sur', 'avec', 'sans',
    'sous', 'entre', 'est', 'sont', 'a', 'ont', 'été', 'être', 'avoir',
    'à la suite', 'au cours', 'au sein', 'en effet', 'par exemple',
})

## 2. Data Classes

In [ ]:
@dataclass
class LinkPrediction:
    """A predicted link."""
    phrase: str
    target_id: int
    target_title: str
    score: float
    probability: float
    linkability: int
    num_targets: int
    start_pos: int = -1
    end_pos: int = -1
    semantic_score: float = 0.0

@dataclass 
class PipelineMetrics:
    """Evaluation metrics."""
    precision: float = 0.0
    recall: float = 0.0
    f1: float = 0.0
    tp: int = 0
    fp: int = 0
    fn: int = 0
    
    def __str__(self):
        return f"P={self.precision:.3f} R={self.recall:.3f} F1={self.f1:.3f}"

## 3. Main Pipeline Class

In [ ]:
class WikipediaLinkPredictor:
    """
    Production Wikipedia link prediction pipeline.
    
    Pipeline stages:
    1. Phrase extraction (n-grams matching anchor dict)
    2. Candidate scoring (V6 linkability-based)
    3. Semantic disambiguation (for ambiguous anchors)
    4. First-occurrence filtering
    """
    
    def __init__(self, config: Config = CONFIG):
        self.config = config
        self._load_resources()
    
    def _load_resources(self):
        """Load all required resources."""
        print("\n📦 Loading resources...")
        t0 = time.time()
        
        # Qdrant
        self.client = QdrantClient(
            host=self.config.qdrant_host,
            port=self.config.qdrant_port,
            prefer_grpc=True,
            timeout=60
        )
        print("   ✅ Qdrant connected")
        
        # Mappings
        with open(self.config.mapping_cache, 'rb') as f:
            data = pickle.load(f)
        self.url_to_id = data['url_to_id']
        self.id_to_title = data['id_to_title']
        print(f"   ✅ Mappings: {len(self.id_to_title):,} articles")
        
        # Anchor dictionary
        with open(self.config.anchor_cache, 'rb') as f:
            self.anchor_dict = pickle.load(f)
        self.anchor_keys = frozenset(self.anchor_dict.keys())
        
        # Precompute totals and linkability
        self.anchor_totals = {}
        self.phrase_linkability = {}
        for phrase, targets in self.anchor_dict.items():
            total = sum(targets.values())
            self.anchor_totals[phrase] = total
            self.phrase_linkability[phrase] = total
        print(f"   ✅ Anchors: {len(self.anchor_dict):,} phrases")
        
        # Lazy load model
        self._model = None
        
        print(f"   ✅ Loaded in {time.time()-t0:.1f}s")
    
    @property
    def model(self):
        """Lazy load embedding model."""
        if self._model is None:
            print("   Loading embedding model...")
            self._model = SentenceTransformer(
                self.config.embedding_model,
                device=self.config.device
            )
            print(f"   ✅ Model loaded on {self.config.device}")
        return self._model
    
    # ========================================================================
    # PHRASE EXTRACTION
    # ========================================================================
    
    def extract_phrases(self, text: str) -> List[Tuple[str, int, int]]:
        """Extract all phrases that exist in anchor dictionary."""
        words = re.findall(r'\b[\w\-\'àâäéèêëïîôùûüçœæ]+\b', text.lower())
        word_positions = [
            (m.start(), m.end()) 
            for m in re.finditer(r'\b[\w\-\'àâäéèêëïîôùûüçœæ]+\b', text.lower())
        ]
        
        phrases = []
        seen = set()
        
        # Longest first for better coverage
        for n in range(self.config.max_ngram, 0, -1):
            for i in range(len(words) - n + 1):
                ngram = ' '.join(words[i:i+n])
                
                if ngram in seen or ngram in STOP_WORDS or len(ngram) <= 1:
                    continue
                
                if ngram in self.anchor_keys:
                    start = word_positions[i][0]
                    end = word_positions[i+n-1][1]
                    phrases.append((ngram, start, end))
                    seen.add(ngram)
        
        return phrases
    
    # ========================================================================
    # SCORING (V6)
    # ========================================================================
    
    def score_candidates(
        self,
        text: str,
        phrases: List[Tuple[str, int, int]],
        article_id: int,
        article_title: str
    ) -> List[LinkPrediction]:
        """Score all candidates using V6 linkability-based scoring."""
        article_title_lower = article_title.lower().strip()
        candidates = []
        
        for phrase, start, end in phrases:
            if phrase == article_title_lower:
                continue
            
            targets = self.anchor_dict[phrase]
            total = self.anchor_totals[phrase]
            linkability = self.phrase_linkability[phrase]
            num_targets = len(targets)
            
            is_cap = text[start].isupper() if start < len(text) else False
            word_count = len(phrase.split())
            
            for target_id, count in targets.items():
                if target_id == article_id:
                    continue
                
                prob = count / total if total > 0 else 0
                score = self._compute_score(
                    prob, linkability, count, num_targets, word_count, is_cap
                )
                
                if score >= self.config.score_threshold:
                    candidates.append(LinkPrediction(
                        phrase=phrase,
                        target_id=target_id,
                        target_title=self.id_to_title.get(target_id, ''),
                        score=score,
                        probability=prob,
                        linkability=linkability,
                        num_targets=num_targets,
                        start_pos=start,
                        end_pos=end,
                    ))
        
        return candidates
    
    def _compute_score(
        self, prob: float, linkability: int, count: int,
        num_targets: int, word_count: int, is_cap: bool
    ) -> float:
        """V6 scoring function."""
        score = prob
        
        # Probability bonuses
        if prob >= 0.9: score *= 2.5
        elif prob >= 0.7: score *= 2.0
        elif prob >= 0.5: score *= 1.5
        elif prob >= 0.3: score *= 1.2
        
        # Linkability (sweet spot: 20-500)
        if linkability >= 100: score *= 1.5
        elif linkability >= 50: score *= 1.3
        elif linkability >= 20: score *= 1.1
        elif linkability >= 5: score *= 1.0
        elif linkability >= 2: score *= 0.7
        else: score *= 0.4
        
        # Count bonus
        if count >= 50: score *= 1.3
        elif count >= 20: score *= 1.2
        elif count >= 5: score *= 1.1
        elif count == 1: score *= 0.6
        
        # Multi-word bonus
        if word_count >= 4: score *= 1.5
        elif word_count >= 3: score *= 1.3
        elif word_count >= 2: score *= 1.15
        
        # Capitalization
        if is_cap: score *= 1.3
        
        # Ambiguity penalty
        if num_targets > 50: score *= 0.6
        elif num_targets > 30: score *= 0.7
        elif num_targets > 20: score *= 0.8
        elif num_targets > 10: score *= 0.9
        
        # Single lowercase penalty
        if word_count == 1 and not is_cap: score *= 0.5
        
        return score
    
    # ========================================================================
    # DISAMBIGUATION
    # ========================================================================
    
    def disambiguate(self, text: str, candidates: List[LinkPrediction]) -> List[LinkPrediction]:
        """Use semantic similarity for ambiguous anchors."""
        if not self.config.use_semantic_disambiguation:
            return candidates
        
        # Group by phrase
        phrase_candidates = defaultdict(list)
        for c in candidates:
            phrase_candidates[c.phrase].append(c)
        
        # Find ambiguous
        ambiguous = [
            p for p, cands in phrase_candidates.items()
            if len(cands) > 1 and cands[0].num_targets >= self.config.disambiguation_threshold
        ]
        
        if not ambiguous:
            return candidates
        
        # Encode context
        context_emb = self.model.encode(
            f"query: {text[:1000]}",
            normalize_embeddings=True,
            convert_to_numpy=True
        )
        
        # Compute similarities
        for phrase in ambiguous:
            cands = phrase_candidates[phrase]
            target_ids = [c.target_id for c in cands]
            
            try:
                points = self.client.retrieve(
                    collection_name=self.config.collection_name,
                    ids=target_ids,
                    with_vectors=True
                )
                id_to_emb = {p.id: np.array(p.vector) for p in points}
                
                for c in cands:
                    emb = id_to_emb.get(c.target_id)
                    if emb is not None:
                        c.semantic_score = float(np.dot(context_emb, emb))
                        c.score *= (1 + c.semantic_score) / 2
            except:
                pass
        
        return [c for cands in phrase_candidates.values() for c in cands]
    
    # ========================================================================
    # DEDUPLICATION
    # ========================================================================
    
    def deduplicate(self, candidates: List[LinkPrediction]) -> List[LinkPrediction]:
        """Keep only first occurrence of each target/phrase."""
        candidates.sort(key=lambda x: (-x.score, x.start_pos))
        
        final = []
        seen_targets, seen_phrases = set(), set()
        
        for c in candidates:
            if c.target_id in seen_targets or c.phrase in seen_phrases:
                continue
            final.append(c)
            seen_targets.add(c.target_id)
            seen_phrases.add(c.phrase)
            if len(final) >= self.config.max_links_per_article:
                break
        
        return final
    
    # ========================================================================
    # MAIN PREDICTION
    # ========================================================================
    
    def predict(
        self, text: str, article_id: int = 0, article_title: str = ""
    ) -> List[LinkPrediction]:
        """Predict links for an article."""
        phrases = self.extract_phrases(text)
        candidates = self.score_candidates(text, phrases, article_id, article_title)
        candidates = self.disambiguate(text, candidates)
        return self.deduplicate(candidates)
    
    # ========================================================================
    # EVALUATION
    # ========================================================================
    
    def evaluate(self, articles: List[Dict], verbose: bool = False) -> PipelineMetrics:
        """Evaluate on articles with ground truth."""
        total_tp, total_fp, total_fn = 0, 0, 0
        
        for article in tqdm(articles, desc="Evaluating", disable=not verbose):
            predictions = self.predict(article['text'], article['id'], article['title'])
            pred_ids = set(p.target_id for p in predictions)
            gt_ids = article['gt']
            
            total_tp += len(pred_ids & gt_ids)
            total_fp += len(pred_ids - gt_ids)
            total_fn += len(gt_ids - pred_ids)
        
        p = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
        r = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
        f1 = 2 * p * r / (p + r) if (p + r) > 0 else 0
        
        return PipelineMetrics(precision=p, recall=r, f1=f1, tp=total_tp, fp=total_fp, fn=total_fn)
    
    # ========================================================================
    # FETCH FROM LOCAL DB
    # ========================================================================
    
    def get_article_by_title(self, title: str) -> Optional[Dict]:
        """Fetch article from local Qdrant database by title."""
        print(f"   Searching for '{title}' in database...")
        
        try:
            # Semantic search for title
            query_emb = self.model.encode(f"query: {title}", normalize_embeddings=True)
            results = self.client.query_points(
                collection_name=self.config.collection_name,
                query=query_emb.tolist(),
                limit=10,
                with_payload=True
            ).points
            
            # Find best match
            for result in results:
                result_title = result.payload.get('title', '')
                if title.lower() == result_title.lower():
                    text = result.payload.get('text') or result.payload.get('text_withoutHref', '')
                    print(f"   ✅ Found exact match: {result_title}")
                    return {
                        'id': result.payload.get('id'),
                        'title': result_title,
                        'text': text
                    }
            
            # Partial match
            for result in results:
                result_title = result.payload.get('title', '')
                if title.lower() in result_title.lower() or result_title.lower() in title.lower():
                    text = result.payload.get('text') or result.payload.get('text_withoutHref', '')
                    print(f"   ✅ Found similar: {result_title}")
                    return {
                        'id': result.payload.get('id'),
                        'title': result_title,
                        'text': text
                    }
            
            print(f"   ❌ Not found")
            return None
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
            return None

## 4. HTML Generation

In [ ]:
def generate_comparison_html(
    title: str,
    text: str,
    predictions: List[LinkPrediction],
    original_html: str = ""
) -> str:
    """Generate Wikipedia-style HTML with predicted links."""
    
    # Insert predicted links (reverse order to preserve positions)
    sorted_preds = sorted(predictions, key=lambda x: x.start_pos, reverse=True)
    predicted_text = text
    
    for pred in sorted_preds:
        if pred.start_pos >= 0 and pred.end_pos > pred.start_pos:
            orig = predicted_text[pred.start_pos:pred.end_pos]
            wiki_title = pred.target_title.replace(" ", "_")
            link = f'<a href="https://fr.wikipedia.org/wiki/{wiki_title}" class="predicted-link" title="{html.escape(pred.target_title)} (score: {pred.score:.1f})">{orig}</a>'
            predicted_text = predicted_text[:pred.start_pos] + link + predicted_text[pred.end_pos:]
    
    # Generate HTML
    return f'''<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <title>{html.escape(title)} - Predicted Links</title>
    <style>
        * {{ box-sizing: border-box; }}
        body {{ 
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif; 
            margin: 0; 
            padding: 20px;
            background: #f8f9fa; 
        }}
        .container {{
            max-width: 960px;
            margin: 0 auto;
            background: white;
            padding: 30px;
            border: 1px solid #a2a9b1;
            border-radius: 2px;
        }}
        h1 {{
            font-family: 'Linux Libertine', Georgia, serif;
            font-size: 1.8em;
            font-weight: normal;
            border-bottom: 1px solid #a2a9b1;
            padding-bottom: 10px;
            margin: 0 0 20px 0;
        }}
        .stats {{
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            padding: 20px;
            border-radius: 8px;
            margin-bottom: 20px;
            display: flex;
            justify-content: space-around;
            text-align: center;
        }}
        .stat-value {{ font-size: 2em; font-weight: bold; }}
        .stat-label {{ font-size: 0.9em; opacity: 0.9; }}
        .content {{
            font-size: 14px;
            line-height: 1.7;
            color: #202122;
        }}
        .predicted-link {{
            color: #0645ad;
            text-decoration: none;
            background: #e8f5e9;
            padding: 1px 3px;
            border-radius: 2px;
            border-bottom: 2px solid #4caf50;
        }}
        .predicted-link:hover {{
            background: #c8e6c9;
            text-decoration: underline;
        }}
        .link-list {{
            margin-top: 30px;
            padding: 20px;
            background: #f8f9fa;
            border-radius: 4px;
        }}
        .link-list h2 {{
            font-size: 1.1em;
            margin: 0 0 15px 0;
            color: #333;
        }}
        .link-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
            gap: 10px;
        }}
        .link-item {{
            font-size: 12px;
            padding: 8px 12px;
            background: white;
            border: 1px solid #ddd;
            border-radius: 4px;
        }}
        .link-item .phrase {{ font-weight: bold; color: #1a73e8; }}
        .link-item .target {{ color: #666; }}
        .link-item .score {{ float: right; color: #4caf50; font-weight: bold; }}
    </style>
</head>
<body>
    <div class="container">
        <h1>{html.escape(title)}</h1>
        
        <div class="stats">
            <div>
                <div class="stat-value">{len(predictions)}</div>
                <div class="stat-label">Links Predicted</div>
            </div>
            <div>
                <div class="stat-value">{sum(1 for p in predictions if p.score >= 5)}</div>
                <div class="stat-label">High Confidence</div>
            </div>
            <div>
                <div class="stat-value">{sum(p.score for p in predictions)/max(len(predictions),1):.1f}</div>
                <div class="stat-label">Avg Score</div>
            </div>
        </div>
        
        <div class="content">
            {predicted_text.replace(chr(10), '<br>')}
        </div>
        
        <div class="link-list">
            <h2>🔗 All Predicted Links (sorted by score)</h2>
            <div class="link-grid">
                {"".join(f'''<div class="link-item">
                    <span class="score">{p.score:.1f}</span>
                    <span class="phrase">{html.escape(p.phrase)}</span><br>
                    <span class="target">→ <a href="https://fr.wikipedia.org/wiki/{p.target_title.replace(" ", "_")}">{html.escape(p.target_title[:40])}</a></span>
                </div>''' for p in sorted(predictions, key=lambda x: -x.score)[:50])}
            </div>
        </div>
    </div>
</body>
</html>'''

## 5. Helper Functions

In [ ]:
def load_test_articles(pipeline, n: int = 100, min_links: int = 5) -> List[Dict]:
    """Load articles with ground truth for evaluation."""
    print(f"\n📂 Loading {n} test articles...")
    t0 = time.time()
    
    articles = []
    offset = None
    
    while len(articles) < n * 3:
        points, offset = pipeline.client.scroll(
            collection_name="wikipedia_fr",
            limit=500, offset=offset,
            with_payload=True, with_vectors=False
        )
        for point in points:
            text = point.payload.get('text') or point.payload.get('text_withoutHref', '')
            if text and len(text) >= 500:
                articles.append({
                    'id': point.payload.get('id'),
                    'title': point.payload.get('title', ''),
                    'text': text
                })
        if offset is None:
            break
    
    # Load ground truth
    article_ids = [a['id'] for a in articles]
    df = pl.scan_parquet(CONFIG.parquet_path).filter(
        pl.col('id').is_in(article_ids)
    ).select(['id', 'links']).collect()
    
    gt_map = {}
    for row in df.iter_rows(named=True):
        article_id = row['id']
        links_raw = row.get('links')
        if links_raw is None:
            gt_map[article_id] = frozenset()
            continue
        targets = set()
        for link in links_raw:
            href = link.get('href_decoded', '')
            if href:
                tid = (pipeline.url_to_id.get(href) or 
                       pipeline.url_to_id.get(href.replace("_", " ")) or 
                       pipeline.url_to_id.get(href.lower()))
                if tid:
                    targets.add(tid)
        gt_map[article_id] = frozenset(targets)
    
    filtered = []
    for a in articles:
        gt = gt_map.get(a['id'], frozenset())
        if len(gt) >= min_links:
            a['gt'] = gt
            filtered.append(a)
        if len(filtered) >= n:
            break
    
    print(f"   ✅ Loaded {len(filtered)} articles ({time.time()-t0:.1f}s)")
    return filtered

## 6. Initialize Pipeline

In [ ]:
pipeline = WikipediaLinkPredictor(CONFIG)

## 7. Evaluate on Test Set

In [ ]:
print("\n" + "="*70)
print("📊 EVALUATION ON TEST SET")
print("="*70)

test_articles = load_test_articles(pipeline, n=100, min_links=5)
metrics = pipeline.evaluate(test_articles, verbose=True)

print(f"\n{'='*50}")
print(f"🏆 RESULTS")
print(f"{'='*50}")
print(f"   Precision: {metrics.precision:.4f}")
print(f"   Recall:    {metrics.recall:.4f}")
print(f"   F1 Score:  {metrics.f1:.4f}")
print(f"   TP: {metrics.tp}, FP: {metrics.fp}, FN: {metrics.fn}")
print(f"{'='*50}")

## 8. Test on Specific Articles from Database

In [ ]:
print("\n" + "="*70)
print("🔬 TESTING ON SPECIFIC ARTICLES")
print("="*70)

test_titles = [
    "Première Guerre mondiale",
    "Marie Curie",
    "Tour Eiffel",
    "Révolution française",
    "Albert Einstein",
    "Paris",
    "France",
]

for title in test_titles:
    print(f"\n{'='*50}")
    print(f"📄 {title}")
    print("="*50)
    
    # Fetch from local database
    article = pipeline.get_article_by_title(title)
    
    if article is None:
        continue
    
    # Predict
    t0 = time.time()
    predictions = pipeline.predict(
        article['text'], 
        article['id'], 
        article['title']
    )
    print(f"   ✅ {len(predictions)} links in {time.time()-t0:.2f}s")
    
    # Top predictions
    print(f"\n   Top 10:")
    for p in predictions[:10]:
        print(f"   • {p.phrase} → {p.target_title[:40]} (s={p.score:.1f})")
    
    # Save HTML
    html_output = generate_comparison_html(article['title'], article['text'], predictions)
    safe_name = re.sub(r'[^\w\-]', '_', article['title'])
    filename = f"predicted_{safe_name}.html"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_output)
    print(f"\n   💾 Saved: {filename}")

## 9. Detailed Sample Analysis

In [ ]:
print("\n" + "="*70)
print("🔬 DETAILED SAMPLE ANALYSIS")
print("="*70)

sample = test_articles[0]
predictions = pipeline.predict(sample['text'], sample['id'], sample['title'])

pred_ids = set(p.target_id for p in predictions)
gt_ids = sample['gt']

print(f"\nArticle: {sample['title']}")
print(f"   GT: {len(gt_ids)}, Predicted: {len(predictions)}")
print(f"   TP: {len(pred_ids & gt_ids)}, FP: {len(pred_ids - gt_ids)}, FN: {len(gt_ids - pred_ids)}")

print(f"\n   ✅ Correct predictions (TP):")
for p in [p for p in predictions if p.target_id in gt_ids][:10]:
    print(f"      '{p.phrase}' → {p.target_title[:40]}")

print(f"\n   ❌ False positives (FP):")
for p in [p for p in predictions if p.target_id not in gt_ids][:10]:
    print(f"      '{p.phrase}' → {p.target_title[:40]}")

## 10. Summary

In [ ]:
print("\n" + "="*70)
print("✅ PIPELINE COMPLETE")
print("="*70)
print(f"""
📊 Final Results:
   • Precision: {metrics.precision:.4f}
   • Recall:    {metrics.recall:.4f}
   • F1 Score:  {metrics.f1:.4f}

📁 Generated Files:
   • predicted_*.html - Article predictions with links

🔧 Configuration:
   • Threshold: {CONFIG.score_threshold}
   • Disambiguation: {CONFIG.use_semantic_disambiguation}
   • Device: {CONFIG.device}
""")